# Data 612 - Project 2 : Content-Based and Collaborative Filtering

#### Team Info 
+ Christina Valore, 
+ Juliann McEachern, 
+ Rajwant Mishra

#### Date: June 18, 2019

## Overview

For assignment 2, start with an existing dataset of user-item ratings, such as our toy books dataset, MovieLens, Jester [http://eigentaste.berkeley.edu/dataset/] or another dataset of your choosing.  Implement at least two of these recommendation algorithms: 
* Content-Based Filtering 
* User-User Collaborative Filtering 
* Item-Item Collaborative Filtering 
 

As an example of implementing a Content-Based recommender, you could build item profiles for a subset of MovieLens movies from scraping http://www.imdb.com/ or using the API at https://www.omdbapi.com/ (which has very recently instituted a small monthly fee).  A more challenging method would be to pull movie summaries or reviews and apply tf-idf and/or topic modeling. You should evaluate and compare different approaches, using different algorithms, normalization techniques, similarity methods, neighborhood sizes, etc.  You don’t need to be exhaustive—these are just some suggested possibilities.   You may use the course text’s [recommenderlab](https://cran.r-project.org/web/packages/recommenderlab/vignettes/recommenderlab.pdf) or any other library that you want.   Please provide at least one graph, and a textual summary of your findings and recommendations. 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load data from local csv  into pandas dataframe

Bk_Rating = pd.read_csv('BX-Book-Ratings.csv')

Bk_Book = pd.read_csv('BX-Books.csv')


b'Skipping line 92: expected 3 fields, saw 5\nSkipping line 312: expected 3 fields, saw 5\nSkipping line 317: expected 3 fields, saw 5\nSkipping line 434: expected 3 fields, saw 4\nSkipping line 535: expected 3 fields, saw 5\nSkipping line 555: expected 3 fields, saw 5\nSkipping line 811: expected 3 fields, saw 4\nSkipping line 837: expected 3 fields, saw 4\nSkipping line 880: expected 3 fields, saw 5\nSkipping line 950: expected 3 fields, saw 5\nSkipping line 951: expected 3 fields, saw 4\nSkipping line 1059: expected 3 fields, saw 5\nSkipping line 1258: expected 3 fields, saw 5\nSkipping line 1287: expected 3 fields, saw 5\nSkipping line 1349: expected 3 fields, saw 5\nSkipping line 1450: expected 3 fields, saw 4\nSkipping line 1457: expected 3 fields, saw 5\nSkipping line 1510: expected 3 fields, saw 4\nSkipping line 1711: expected 3 fields, saw 5\nSkipping line 1818: expected 3 fields, saw 4\nSkipping line 1910: expected 3 fields, saw 5\nSkipping line 2030: expected 3 fields, saw 5

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [9]:
Bk_User = pd.read_csv('BX-Users.csv',sep= ';')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2
